In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import operator
import time

%matplotlib inline

members = pd.read_csv('../Data Files/NSS_GreatCare_Members.csv', parse_dates=[61,62])
admissions = pd.read_csv('../Data Files/NSS_Hospital_Inpatient_Admissions.csv', parse_dates=[8,9])
pcp_visits = pd.read_csv('../Data Files/NSS_PrimaryCare_Office_Visits.csv', parse_dates=[12])

In [2]:
GCM = pd.read_csv('Data Files/NSS_GreatCare_Members.csv')

In [3]:
GCM.head()

,Patient ID,Primary Plan ID,Primary Payor ID,Is Patient Alive,Patient Age Integer,Patient Date of Birth,Patient ZIP Code 5-Digit,Count Primary Care Visits in Last 36 Months,Count Visits with Attributee Individual Primary Care Provider i,Count Visits with Attributee Primary Care Provider Group in Las,...,Is CCW Prostate Cancer,Is CCW Rheumatoid Arthritis or Osteoarthritis,Is CCW Stroke or Transient Ischemic Attack,Is CCW ASCVD,Is CCW Anxiety Disorders,Is CCW Depressive Disorders,Is CCW Schizophrenia or Other Psychotic Disorders,Is CCW Bipolar Disorder,Period Start Date,Period End Date
0,DS1|1,DS1|5047W,DS1|1,1,54,1958-11-08,92111.0,0,NaN,NaN,...,0,0,0,0,0,0,0,0,2013-01-01 00:00:00+00,2013-04-11 00:00:00+00
1,DS1|10,DS1|5047W,DS1|1,1,54,1958-12-10,90024.0,6,1.0,5.0,...,0,0,0,0,1,0,0,0,2012-12-10 00:00:00+00,2013-01-01 00:00:00+00
2,DS1|100,DS1|5047W,DS1|1,1,2,2012-08-04,90008.0,1,1.0,1.0,...,0,0,0,0,0,0,0,0,2015-04-18 00:00:00+00,2015-05-18 00:00:00+00
3,DS1|1000,DS1|5047W,DS1|1,1,6,2003-12-25,94131.0,0,NaN,NaN,...,0,0,0,0,0,0,0,0,2010-01-01 00:00:00+00,2010-04-09 00:00:00+00
4,DS1|10000,DS1|5047W,DS1|1,1,7,2005-07-15,90301.0,0,NaN,NaN,...,0,0,0,0,0,0,0,0,2012-08-01 00:00:00+00,2012-10-02 00:00:00+00


In [4]:
HIA = pd.read_csv('Data Files/NSS_Hospital_Inpatient_Admissions.csv')

In [5]:
HIA.head()

,Encounter ID,Patient ID,Encounter Primary Payor ID,Encounter Primary Plan ID,Facility Provider ID,Attending Provider ID,Attending Provider NPI,Facility Provider NPI,Encounter Start Date,Encounter End Date,...,CMS Admit Source Code,CMS Patient Discharge Status Code,CMS Place of Service Code,Principal Discharge Diagnosis ICD-9-CM Code,Principal Discharge Diagnosis ICD-9-CM Description,Principal Discharge Diagnosis ICD-10-CM Code,Principal Discharge Diagnosis ICD-10-CM Description,Primary Plan Allowed Amount,Primary Plan Paid Amount,Patient Paid Amount
0,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,DS1|1,DS1|5047W,DS1|NPI|1306890389,DS1|NPI|1417170523,1.417171e+09,1306890389,2013-01-12,2013-01-14,...,1,1,21.0,NaN,NaN,O70.0,First degree perineal laceration during delivery,0,6620.72,250.0
1,HIA|DS1|10043|DS1|NPI|1902803315|1,DS1|10043,DS1|1,DS1|5047Y,DS1|NPI|1902803315,DS1|NPI|1669599197,1.669599e+09,1902803315,2014-01-05,2014-01-12,...,1,1,21.0,348.4,Compression of brain,NaN,NaN,0,46465.40,250.0
2,HIA|DS1|10073|DS1|NPI|1902803315|1,DS1|10073,DS1|1,DS1|5047X,DS1|NPI|1902803315,DS1|NPI|1407880511,1.407881e+09,1902803315,2012-06-13,2012-06-21,...,1,1,21.0,NaN,NaN,K91.71,Accidental puncture and laceration of a digest...,0,43879.13,250.0
3,HIA|DS1|10101|DS1|NPI|1427055839|1,DS1|10101,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1407008907,1.407009e+09,1427055839,2012-12-06,2012-12-09,...,1,1,21.0,NaN,NaN,O70.1,Second degree perineal laceration during delivery,0,14800.00,250.0
4,HIA|DS1|10118|DS1|NPI|1992818256|1,DS1|10118,DS1|1,DS1|5047W,DS1|NPI|1992818256,DS1|NPI|1518045608,1.518046e+09,1992818256,2015-03-11,2015-03-14,...,2,1,NaN,NaN,NaN,F32.9,"Major depressive disorder, single episode, uns...",0,0.00,0.0


In [6]:
HIA[HIA['Patient ID'] == 'DS1|9994']

,Encounter ID,Patient ID,Encounter Primary Payor ID,Encounter Primary Plan ID,Facility Provider ID,Attending Provider ID,Attending Provider NPI,Facility Provider NPI,Encounter Start Date,Encounter End Date,...,CMS Admit Source Code,CMS Patient Discharge Status Code,CMS Place of Service Code,Principal Discharge Diagnosis ICD-9-CM Code,Principal Discharge Diagnosis ICD-9-CM Description,Principal Discharge Diagnosis ICD-10-CM Code,Principal Discharge Diagnosis ICD-10-CM Description,Primary Plan Allowed Amount,Primary Plan Paid Amount,Patient Paid Amount
6210,HIA|DS1|9994|DS1|NPI|1689608150|1,DS1|9994,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1861831836,1.861832e+09,1689608150,2015-11-21,2015-11-23,...,1,1,21.0,NaN,NaN,O76,Abnormality in fetal heart rate and rhythm com...,0,17919.75,250.0
6211,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1750544516,1.750545e+09,1689608150,2015-11-24,2015-11-27,...,1,1,21.0,NaN,NaN,O42.92,"Full-term premature rupture of membranes, unsp...",0,19919.42,250.0


In [7]:
HIA.dtypes

Encounter ID                                            object
Patient ID                                              object
Encounter Primary Payor ID                              object
Encounter Primary Plan ID                               object
Facility Provider ID                                    object
Attending Provider ID                                   object
Attending Provider NPI                                 float64
Facility Provider NPI                                   object
Encounter Start Date                                    object
Encounter End Date                                      object
Is Emergency Department Visit Parent Encounter           int64
Is Observation Stay Parent Encounter                     int64
CMS Admit Type Code                                    float64
CMS Admit Source Code                                   object
CMS Patient Discharge Status Code                        int64
CMS Place of Service Code                              

In [8]:
HIA = HIA.astype({'Encounter Start Date':'datetime64[ns]',
                                'Encounter End Date':'datetime64[ns]'})

In [9]:
HIA.dtypes

Encounter ID                                                   object
Patient ID                                                     object
Encounter Primary Payor ID                                     object
Encounter Primary Plan ID                                      object
Facility Provider ID                                           object
Attending Provider ID                                          object
Attending Provider NPI                                        float64
Facility Provider NPI                                          object
Encounter Start Date                                   datetime64[ns]
Encounter End Date                                     datetime64[ns]
Is Emergency Department Visit Parent Encounter                  int64
Is Observation Stay Parent Encounter                            int64
CMS Admit Type Code                                           float64
CMS Admit Source Code                                          object
CMS Patient Discharg

In [10]:
HIA_dedup = HIA.sort_values(by=['Patient ID','Encounter Start Date', 'Encounter End Date']).drop_duplicates(subset=['Patient ID', 'Encounter Start Date'], keep = 'last')
HIA_dedup = HIA_dedup.sort_values(by=['Patient ID','Encounter End Date', 'Encounter Start Date']).drop_duplicates(subset=['Patient ID', 'Encounter End Date'], keep = 'first')
HIA.shape, HIA_dedup.shape

((6213, 23), (6149, 23))

In [11]:
total_HIA = HIA.shape[0]

In [12]:
PCOV = pd.read_csv('Data Files/NSS_PrimaryCare_Office_Visits.csv')

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (3,5,17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [13]:
PCOV = PCOV.astype({'Encounter Date':'datetime64[ns]'})

In [14]:
PCOV.head()

,Encounter ID,Patient ID,Service Provider ID,Attending Provider ID,Provider Group Provider ID,Facility Provider ID,Encounter Primary Payor ID,Encounter Primary Plan ID,Attending Provider NPI,Facility Provider NPI,Service Provider NPI,Provider Group NPI,Encounter Date,Is Qualifying Preventive Primary Care Service,Is Qualifying Primary Care Service Provider,Is Qualifying Primary Care Attending,Is Qualifying Primary Care Provider Group,Qualifying Preventive Primary Care HCPCS Code
0,COV|EE|DS1|10076|20110225|DS1|NPI|1487856779,DS1|10076,DS1|NPI|1487856779,NaN,DS1|NPI|1043427248,NaN,DS1|1,DS1|5047W,NaN,NaN,1.487857e+09,1.043427e+09,2011-02-18,0,1,0,0,NaN
1,COV|EE|DS1|10088|20140222|DS1|NPI|1528030996,DS1|10088,DS1|NPI|1528030996,DS1|NPI|1528030996,DS1|NPI|1043427248,DS1|NPI|1902803315,DS1|1,DS1|5047W,1.528031e+09,1.902803e+09,1.528031e+09,1.043427e+09,2014-02-22,0,1,1,1,NaN
2,COV|EE|DS1|10165|20140907|DS1|NPI|1477579365,DS1|10165,DS1|NPI|1477579365,NaN,DS1|NPI|1043427248,NaN,DS1|1,DS1|5047W,NaN,NaN,1.477579e+09,1.043427e+09,2014-08-31,0,1,0,0,NaN
3,COV|EE|DS1|10240|20130624|DS1|NPI|1821178088,DS1|10240,DS1|NPI|1821178088,DS1|NPI|1821178088,DS1|NPI|1407058787,DS1|NPI|1104842020,DS1|1,DS1|5047W,1.821178e+09,1.104842e+09,1.821178e+09,1.407059e+09,2013-06-17,0,1,1,1,NaN
4,COV|EE|DS1|10293|20160904|DS1|NPI|1609004191,DS1|10293,DS1|NPI|1609004191,NaN,DS1|NPI|1184722779,NaN,DS1|1,DS1|5047X,NaN,NaN,1.609004e+09,1.184723e+09,2016-09-04,0,1,0,0,NaN


In [15]:
PCOV.dtypes

Encounter ID                                             object
Patient ID                                               object
Service Provider ID                                      object
Attending Provider ID                                    object
Provider Group Provider ID                               object
Facility Provider ID                                     object
Encounter Primary Payor ID                               object
Encounter Primary Plan ID                                object
Attending Provider NPI                                  float64
Facility Provider NPI                                   float64
Service Provider NPI                                    float64
Provider Group NPI                                      float64
Encounter Date                                   datetime64[ns]
Is Qualifying Preventive Primary Care Service             int64
Is Qualifying Primary Care Service Provider               int64
Is Qualifying Primary Care Attending    

hia = HIA.astype(object)
hia.head(10)
hia.info()
hia2 = HIA['Attending Provider NPI'].astype(object)
hia2.head()
hia2
HIA.drop(['Encounter ID'])

In [16]:
def categorize_events(patient_admissions):
    patient_index_events = 0
    patient_readmission_events = 0
    last_index_event_discharge_date = None
    last_index_event_encounter_id = None
    readmission = False
    
    for ind, row in patient_admissions.iterrows():
        if patient_index_events == 0 or row['Encounter Start Date'] > last_index_event_discharge_date + dt.timedelta(days=30):
            patient_index_events += 1
            last_index_event_discharge_date = row['Encounter End Date']
            last_index_event_encounter_id = row['Encounter ID']
            readmission = False
            HIA.loc[HIA['Encounter ID'] == row['Encounter ID'], 'type'] = 'index without readmission'
        elif readmission == False:
            patient_readmission_events += 1
            readmission = True
            HIA.loc[HIA['Encounter ID'] == last_index_event_encounter_id, 'type'] = 'index with readmission'
            HIA.loc[HIA['Encounter ID'] == row['Encounter ID'], 'type'] = 'initial readmission'
        else:
            HIA.loc[HIA['Encounter ID'] == row['Encounter ID'], 'type'] = 'extra readmission'
            
    return patient_index_events, patient_readmission_events

In [17]:
total_index_events = 0
total_index_events_leading_to_readmission = 0

for patient in HIA_dedup['Patient ID'].unique():
    patient_admissions = HIA_dedup[HIA['Patient ID']==patient]
    times_admitted = patient_admissions.shape[0]
    if times_admitted > 1:
        patient_admissions.sort_values(by = 'Encounter End Date', inplace=True, ignore_index=True)
        ind, re = categorize_events(patient_admissions)
        total_index_events += ind
        total_index_events_leading_to_readmission += re
    else:
        encounter_id = patient_admissions['Encounter ID'].values[0]
        HIA.loc[HIA['Encounter ID'] == encounter_id, 'type'] = 'index without readmission'
        total_index_events += 1
print("Total Index Events:",total_index_events) 
print("Index Events Leading to Readmission:", total_index_events_leading_to_readmission)
print(str.format("Readmission Rate: {}%",round(total_index_events_leading_to_readmission/total_index_events*100,2)))

<ipython-input-17-2189fea52b40>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  patient_admissions = HIA_dedup[HIA['Patient ID']==patient]
<ipython-input-17-2189fea52b40>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patient_admissions.sort_values(by = 'Encounter End Date', inplace=True, ignore_index=True)


Total Index Events: 5536
Index Events Leading to Readmission: 527
Readmission Rate: 9.52%


def categorize_events(patient_admissions):
    patient_index_events = 0
    patient_readmission_events = 0
    last_index_event_discharge_date = None
    readmission = False
    for ind, row in patient_admissions.iterrows():
        if patient_index_events == 0 or row['Encounter Date'] > last_index_event_discharge_date + dt.timedelta(days=7):
            patient_index_events += 1
            last_index_event_discharge_date = row['Encounter Date']
            readmission = False
            PCOV.loc[PCOV['Encounter ID'] == row['Encounter ID'], 'type'] = 'index'
        elif readmission == False:
            patient_readmission_events += 1
            readmission = True
            PCOV.loc[PCOV['Encounter ID'] == row['Encounter ID'], 'type'] = 'readmission'
        else:
            PCOV.loc[PCOV['Encounter ID'] == row['Encounter ID'], 'type'] = 'readmission'
            
    return patient_index_events, patient_readmission_events

(DOESNT WORK)

total_index_events = 0
total_index_events_leading_to_readmission = 0

for patient in PCOV['Patient ID'].unique():
    patient_admissions = PCOV[PCOV['Patient ID']==patient]
    times_admitted = patient_admissions.shape[0]
    if times_admitted > 1:
        patient_admissions.sort_values(by = 'Encounter Date', inplace=True, ignore_index=True)
        ind, re = categorize_events(patient_admissions)
        total_index_events += ind
        total_index_events_leading_to_readmission += re
    else:
        PCOV.loc[PCOV['Patient ID'] == patient, 'type'] = 'index'
        total_index_events += 1
print("Total Index Events:",total_index_events) 
print("Index Events Leading to Readmission:", total_index_events_leading_to_readmission)
print(str.format("Readmission Rate: {}%",round(total_index_events_leading_to_readmission/total_index_events*100,2)))

(VERY VERY BROKEN TRIED TO RUN FOR AN HOUR AND A HALF)

In [18]:
HIA['type'].value_counts()

index without readmission    5009
index with readmission        527
initial readmission           527
extra readmission              86
Name: type, dtype: int64

In [19]:
HIA[['type']] = HIA[['type']].fillna(value='duplicate encounter')

In [20]:
HIA['type'].value_counts()

index without readmission    5009
index with readmission        527
initial readmission           527
extra readmission              86
duplicate encounter            64
Name: type, dtype: int64

In [21]:
total_hospital_visits = 0
total_pcp_visits_in_7_days = 0
for ind, row in HIA_dedup.iterrows():
    
    total_hospital_visits += 1
    patient = row['Patient ID']
    discharge_date = row['Encounter End Date']
    
    for i,r in PCOV[PCOV['Patient ID']==patient].iterrows():
        if r['Encounter Date'] <= discharge_date + dt.timedelta(days=7) and r['Encounter Date'] >= discharge_date:
            total_pcp_visits_in_7_days += 1
            break


print('Total Hospital Visits: ', total_hospital_visits)
print('Total PCP Visits in 7 Days: ', total_pcp_visits_in_7_days)
print(str.format("PCP Visit Rate: {}%",round(total_pcp_visits_in_7_days/total_hospital_visits*100,2)))

Total Hospital Visits:  6149
Total PCP Visits in 7 Days:  1860
PCP Visit Rate: 30.25%


no_pcp = 0
mult_pcp = 0
one_pcp = 0

for ind, row in HIA_dedup.iterrows():
    id pcp_visits

In [22]:
for ind, row in HIA.iterrows():
    patient = row['Patient ID']
    discharge = row['Encounter End Date']
    encounter = row['Encounter ID']
    mask = (PCOV['Patient ID'] == patient) & (PCOV['Encounter Date']>= discharge)
    patient_pcp = PCOV[mask].sort_values('Encounter Date')
    if len(patient_pcp['Encounter Date']) == 0:
        continue
    else:
        first_pcp_visit = patient_pcp.head(1)
        indiv_npi = first_pcp_visit['Service Provider NPI'].values[0]
        group_npi = first_pcp_visit['Provider Group NPI'].values[0]
        if indiv_npi == group_npi:
            HIA.loc[HIA['Encounter ID'] == encounter, 'individual_npi'] = indiv_npi
        else:
            HIA.loc[HIA['Encounter ID'] == encounter, 'individual_npi'] = indiv_npi
            HIA.loc[HIA['Encounter ID'] == encounter, 'group_npi'] = group_npi

endpoint = 'https://npiregistry.cms.hhs.gov/api/?version=2.0'
params = {'number':1982667663}
response = requests.get(endpoint, params)
response.json()